# Downloading Databases

This ipython notebook downloads data. One file per state containing health, behavioral, and environmental dataper year is downloaded from www.countyhealthrankings.org to the folder "downloads". The script also downloads the suicide rate from www.cdc.gov.

The data from County Health Ranking is tricky to download. The file name tend to be slightly different from year to year and state to state. The name of the files are not available on the webpages where they are linked. This code therefore uses a "guessing" approach which is highly inneficient. A sleeping command is executed in-between every download to avoid being blacklisted by the website. Reduce this time at your own risk. It may take several hours to download all files with the name "guessing" and the default 3 s wait time. To do:
- allow to download the files from a "know file" list (this list is ont he repository already)
- optimize the download process to reduce the "guessing"

Francois Aubin francois.y.aubin@gmail.com

Optimized for Python 3.6 on Ubuntu 18.04

Last update in July 2018

## Some Functions

In [1]:
import os
import time
import requests
import pycurl


def create_data_structure():
    """
    ensure a downloads folder exist and returns the list of files in it
    """
    if 'downloads' not in os.listdir('.'):
        os.mkdir('downloads')
        file_list = []
    else:
        file_list = os.listdir('downloads')
    return file_list


def clear_data_structure():
    """
    erase the downloads folder to start fresh
    """
    if 'downloads' in os.listdir('.'):
        os.system('rm -r downloads')
    return


def guess_names(file_name):
    """
    return a list of potential file names to download
    """
    name = file_name[:-9]

    possible_names = []
    # for v in range(9, 0, -1):
    for v in range(6, 0, -1):
        for ext in ['', '_0', '_1']:
            for pre in ['%20', '']:
                for rank in ['Rankings', 'Ranking']:
                    new_name = '{0:s}{1:s}v{2:d}{3:s}.xls'.format(name, pre, v,
                                                                  ext)
                    if not rank == 'Rankings':
                        new_name = new_name.replace('Rankings', rank)
                    possible_names.append(new_name)
    return possible_names


def download_file(base_url, file_name, save_file_name, file_size_lowerlim=0,
                  sleep_time=3):
    """
    download a file, save it as save_file_name, erase it if it is smaller than
    file_size_lower_limit (in Bytes)
    """
    url = os.path.join(base_url, file_name)
    print(file_name)
    print('Downloading {0:s}'.format(save_file_name))

    file_ = open(os.path.join('downloads', save_file_name), 'wb')
    curl = pycurl.Curl()
    curl.setopt(pycurl.URL, url)
    curl.setopt(pycurl.FOLLOWLOCATION, 1)
    curl.setopt(pycurl.MAXREDIRS, 5)
    curl.setopt(pycurl.CONNECTTIMEOUT, 30)
    curl.setopt(pycurl.TIMEOUT, 300)
    curl.setopt(pycurl.NOSIGNAL, 1)
    curl.setopt(pycurl.WRITEDATA, file_)
    curl.perform()
    file_.close()
    time.sleep(sleep_time)

    file_size = os.path.getsize(os.path.join('downloads', save_file_name))
    if file_size < file_size_lowerlim:
        print('File size is {0:d} B'.format(file_size))
        os.system("rm downloads/{0:s}".format(save_file_name.replace(' ',
                                                                     '/ ')))
        return False
    else:
        f = open("downloaded.txt", 'a')
        f.write(file_name + '\n')
        f.close()
        return True

## The Script

In [2]:
sleep_time = 0.1  # takes about 3 hours
downloaded_files = create_data_structure()

# suicide data
for year in [2005, 2014, 2015, 2016]:
    base_url = 'https://www.cdc.gov/nchs/pressroom/sosmap/suicide-mortality/'
    url_file_name = 'SUICIDE{0:d}.csv'.format(year)
    if url_file_name not in downloaded_files:
        success = download_file(base_url, url_file_name, url_file_name,
                                file_size_lowerlim=1000,
                                sleep_time=sleep_time)
        if not success:
            exit("Error downloading suicide data")

SUICIDE2005.csv
SUICIDE2014.csv
SUICIDE2015.csv
SUICIDE2016.csv


In [4]:
# health, behavioral, and environmental data
base_url = os.path.join("http://www.countyhealthrankings.org", "sites",
                        "default", "files", "state", "downloads")
years = range(2010, 2019)
states = ['Alabama', 'Alaska', 'Arizona', 'Arkansas', 'California', 'Colorado',
          'Connecticut', 'Delaware', 'District of Columbia', 'Florida',
          'Georgia', 'Hawaii', 'Idaho', 'Illinois', 'Indiana', 'Iowa',
          'Kansas', 'Kentucky', 'Louisiana', 'Maine', 'Maryland',
          'Massachusetts', 'Michigan', 'Minnesota', 'Missouri', 'Montana',
          'Nebraska', 'New Hampshire', 'New Jersey', 'New Mexico', 'New York',
          'North Carolina', 'North Dakota', 'Ohio', 'Oklahoma', 'Oregon',
          'Pennsylvania', 'Rhode Island', 'South Carolina', 'South Dakota',
          'Tennessee', 'Texas', 'Utah', 'Vermont', 'Virginia', 'Washington',
          'West Virginia', 'Wisconsin', 'Wyoming']
for state in states:
    for year in years:
        print(state, year)
        file_name = "{0:d}%20County%20Health%20Rankings".format(year)
        url_state = state.replace(' ',   '%20')
        file_name += "%20{0:s}%20Data%20-%20v1.xls".format(url_state)

        save_file_name = '{0:s}_{1:d}.xls'.format(state, year)

        if save_file_name not in downloaded_files:
            print('---')
            possible_url_file_names = guess_names(file_name)
            for url_file_name in possible_url_file_names:
                success = download_file(base_url, url_file_name,
                                        save_file_name,
                                        file_size_lowerlim=29000,
                                        sleep_time=sleep_time)
                if success:
                    break

Alabama 2010
---
2010%20County%20Health%20Rankings%20Alabama%20Data%20-%20v6.xls
File size is 24096 B
2010%20County%20Health%20Ranking%20Alabama%20Data%20-%20v6.xls
File size is 24095 B
2010%20County%20Health%20Rankings%20Alabama%20Data%20-v6.xls
File size is 24093 B
2010%20County%20Health%20Ranking%20Alabama%20Data%20-v6.xls
File size is 24092 B
2010%20County%20Health%20Rankings%20Alabama%20Data%20-%20v6_0.xls
File size is 24098 B
2010%20County%20Health%20Ranking%20Alabama%20Data%20-%20v6_0.xls
File size is 24097 B
2010%20County%20Health%20Rankings%20Alabama%20Data%20-v6_0.xls
File size is 24095 B
2010%20County%20Health%20Ranking%20Alabama%20Data%20-v6_0.xls
File size is 24094 B
2010%20County%20Health%20Rankings%20Alabama%20Data%20-%20v6_1.xls
File size is 24098 B
2010%20County%20Health%20Ranking%20Alabama%20Data%20-%20v6_1.xls
File size is 24097 B
2010%20County%20Health%20Rankings%20Alabama%20Data%20-v6_1.xls
File size is 24095 B
2010%20County%20Health%20Ranking%20Alabama%20Data%20-v

File size is 24095 B
2011%20County%20Health%20Ranking%20Alabama%20Data%20-v5_1.xls
File size is 24094 B
2011%20County%20Health%20Rankings%20Alabama%20Data%20-%20v4.xls
File size is 24096 B
2011%20County%20Health%20Ranking%20Alabama%20Data%20-%20v4.xls
File size is 24095 B
2011%20County%20Health%20Rankings%20Alabama%20Data%20-v4.xls
File size is 24093 B
2011%20County%20Health%20Ranking%20Alabama%20Data%20-v4.xls
File size is 24092 B
2011%20County%20Health%20Rankings%20Alabama%20Data%20-%20v4_0.xls
File size is 24098 B
2011%20County%20Health%20Ranking%20Alabama%20Data%20-%20v4_0.xls
File size is 24097 B
2011%20County%20Health%20Rankings%20Alabama%20Data%20-v4_0.xls
File size is 24095 B
2011%20County%20Health%20Ranking%20Alabama%20Data%20-v4_0.xls
File size is 24094 B
2011%20County%20Health%20Rankings%20Alabama%20Data%20-%20v4_1.xls
File size is 24098 B
2011%20County%20Health%20Ranking%20Alabama%20Data%20-%20v4_1.xls
File size is 24097 B
2011%20County%20Health%20Rankings%20Alabama%20Data%

File size is 24097 B
2012%20County%20Health%20Rankings%20Alabama%20Data%20-v3_1.xls
File size is 24095 B
2012%20County%20Health%20Ranking%20Alabama%20Data%20-v3_1.xls
File size is 24094 B
2012%20County%20Health%20Rankings%20Alabama%20Data%20-%20v2.xls
File size is 24096 B
2012%20County%20Health%20Ranking%20Alabama%20Data%20-%20v2.xls
Alabama 2013
---
2013%20County%20Health%20Rankings%20Alabama%20Data%20-%20v6.xls
File size is 24096 B
2013%20County%20Health%20Ranking%20Alabama%20Data%20-%20v6.xls
File size is 24095 B
2013%20County%20Health%20Rankings%20Alabama%20Data%20-v6.xls
File size is 24093 B
2013%20County%20Health%20Ranking%20Alabama%20Data%20-v6.xls
File size is 24092 B
2013%20County%20Health%20Rankings%20Alabama%20Data%20-%20v6_0.xls
File size is 24098 B
2013%20County%20Health%20Ranking%20Alabama%20Data%20-%20v6_0.xls
File size is 24097 B
2013%20County%20Health%20Rankings%20Alabama%20Data%20-v6_0.xls
File size is 24095 B
2013%20County%20Health%20Ranking%20Alabama%20Data%20-v6_0.

File size is 24097 B
2015%20County%20Health%20Rankings%20Alabama%20Data%20-v6_0.xls
File size is 24095 B
2015%20County%20Health%20Ranking%20Alabama%20Data%20-v6_0.xls
File size is 24094 B
2015%20County%20Health%20Rankings%20Alabama%20Data%20-%20v6_1.xls
File size is 24098 B
2015%20County%20Health%20Ranking%20Alabama%20Data%20-%20v6_1.xls
File size is 24097 B
2015%20County%20Health%20Rankings%20Alabama%20Data%20-v6_1.xls
File size is 24095 B
2015%20County%20Health%20Ranking%20Alabama%20Data%20-v6_1.xls
File size is 24094 B
2015%20County%20Health%20Rankings%20Alabama%20Data%20-%20v5.xls
File size is 24096 B
2015%20County%20Health%20Ranking%20Alabama%20Data%20-%20v5.xls
File size is 24095 B
2015%20County%20Health%20Rankings%20Alabama%20Data%20-v5.xls
File size is 24093 B
2015%20County%20Health%20Ranking%20Alabama%20Data%20-v5.xls
File size is 24092 B
2015%20County%20Health%20Rankings%20Alabama%20Data%20-%20v5_0.xls
File size is 24098 B
2015%20County%20Health%20Ranking%20Alabama%20Data%20-

File size is 24098 B
2017%20County%20Health%20Ranking%20Alabama%20Data%20-%20v6_0.xls
File size is 24097 B
2017%20County%20Health%20Rankings%20Alabama%20Data%20-v6_0.xls
File size is 24095 B
2017%20County%20Health%20Ranking%20Alabama%20Data%20-v6_0.xls
File size is 24094 B
2017%20County%20Health%20Rankings%20Alabama%20Data%20-%20v6_1.xls
File size is 24098 B
2017%20County%20Health%20Ranking%20Alabama%20Data%20-%20v6_1.xls
File size is 24097 B
2017%20County%20Health%20Rankings%20Alabama%20Data%20-v6_1.xls
File size is 24095 B
2017%20County%20Health%20Ranking%20Alabama%20Data%20-v6_1.xls
File size is 24094 B
2017%20County%20Health%20Rankings%20Alabama%20Data%20-%20v5.xls
File size is 24096 B
2017%20County%20Health%20Ranking%20Alabama%20Data%20-%20v5.xls
File size is 24095 B
2017%20County%20Health%20Rankings%20Alabama%20Data%20-v5.xls
File size is 24093 B
2017%20County%20Health%20Ranking%20Alabama%20Data%20-v5.xls
File size is 24092 B
2017%20County%20Health%20Rankings%20Alabama%20Data%20-

...

File size is 24095 B
2011%20County%20Health%20Ranking%20Wyoming%20Data%20-v5_1.xls
File size is 24094 B
2011%20County%20Health%20Rankings%20Wyoming%20Data%20-%20v4.xls
File size is 24096 B
2011%20County%20Health%20Ranking%20Wyoming%20Data%20-%20v4.xls
File size is 24095 B
2011%20County%20Health%20Rankings%20Wyoming%20Data%20-v4.xls
File size is 24093 B
2011%20County%20Health%20Ranking%20Wyoming%20Data%20-v4.xls
File size is 24092 B
2011%20County%20Health%20Rankings%20Wyoming%20Data%20-%20v4_0.xls
File size is 24098 B
2011%20County%20Health%20Ranking%20Wyoming%20Data%20-%20v4_0.xls
File size is 24097 B
2011%20County%20Health%20Rankings%20Wyoming%20Data%20-v4_0.xls
File size is 24095 B
2011%20County%20Health%20Ranking%20Wyoming%20Data%20-v4_0.xls
File size is 24094 B
2011%20County%20Health%20Rankings%20Wyoming%20Data%20-%20v4_1.xls
File size is 24098 B
2011%20County%20Health%20Ranking%20Wyoming%20Data%20-%20v4_1.xls
File size is 24097 B
2011%20County%20Health%20Rankings%20Wyoming%20Data%

File size is 24097 B
2012%20County%20Health%20Rankings%20Wyoming%20Data%20-v3_1.xls
File size is 24095 B
2012%20County%20Health%20Ranking%20Wyoming%20Data%20-v3_1.xls
File size is 24094 B
2012%20County%20Health%20Rankings%20Wyoming%20Data%20-%20v2.xls
File size is 24096 B
2012%20County%20Health%20Ranking%20Wyoming%20Data%20-%20v2.xls
Wyoming 2013
---
2013%20County%20Health%20Rankings%20Wyoming%20Data%20-%20v6.xls
File size is 24096 B
2013%20County%20Health%20Ranking%20Wyoming%20Data%20-%20v6.xls
File size is 24095 B
2013%20County%20Health%20Rankings%20Wyoming%20Data%20-v6.xls
File size is 24093 B
2013%20County%20Health%20Ranking%20Wyoming%20Data%20-v6.xls
File size is 24092 B
2013%20County%20Health%20Rankings%20Wyoming%20Data%20-%20v6_0.xls
File size is 24098 B
2013%20County%20Health%20Ranking%20Wyoming%20Data%20-%20v6_0.xls
File size is 24097 B
2013%20County%20Health%20Rankings%20Wyoming%20Data%20-v6_0.xls
File size is 24095 B
2013%20County%20Health%20Ranking%20Wyoming%20Data%20-v6_0.

File size is 24097 B
2015%20County%20Health%20Rankings%20Wyoming%20Data%20-v6_0.xls
File size is 24095 B
2015%20County%20Health%20Ranking%20Wyoming%20Data%20-v6_0.xls
File size is 24094 B
2015%20County%20Health%20Rankings%20Wyoming%20Data%20-%20v6_1.xls
File size is 24098 B
2015%20County%20Health%20Ranking%20Wyoming%20Data%20-%20v6_1.xls
File size is 24097 B
2015%20County%20Health%20Rankings%20Wyoming%20Data%20-v6_1.xls
File size is 24095 B
2015%20County%20Health%20Ranking%20Wyoming%20Data%20-v6_1.xls
File size is 24094 B
2015%20County%20Health%20Rankings%20Wyoming%20Data%20-%20v5.xls
File size is 24096 B
2015%20County%20Health%20Ranking%20Wyoming%20Data%20-%20v5.xls
File size is 24095 B
2015%20County%20Health%20Rankings%20Wyoming%20Data%20-v5.xls
File size is 24093 B
2015%20County%20Health%20Ranking%20Wyoming%20Data%20-v5.xls
File size is 24092 B
2015%20County%20Health%20Rankings%20Wyoming%20Data%20-%20v5_0.xls
File size is 24098 B
2015%20County%20Health%20Ranking%20Wyoming%20Data%20-

File size is 24098 B
2017%20County%20Health%20Ranking%20Wyoming%20Data%20-%20v6_0.xls
File size is 24097 B
2017%20County%20Health%20Rankings%20Wyoming%20Data%20-v6_0.xls
File size is 24095 B
2017%20County%20Health%20Ranking%20Wyoming%20Data%20-v6_0.xls
File size is 24094 B
2017%20County%20Health%20Rankings%20Wyoming%20Data%20-%20v6_1.xls
File size is 24098 B
2017%20County%20Health%20Ranking%20Wyoming%20Data%20-%20v6_1.xls
File size is 24097 B
2017%20County%20Health%20Rankings%20Wyoming%20Data%20-v6_1.xls
File size is 24095 B
2017%20County%20Health%20Ranking%20Wyoming%20Data%20-v6_1.xls
File size is 24094 B
2017%20County%20Health%20Rankings%20Wyoming%20Data%20-%20v5.xls
File size is 24096 B
2017%20County%20Health%20Ranking%20Wyoming%20Data%20-%20v5.xls
File size is 24095 B
2017%20County%20Health%20Rankings%20Wyoming%20Data%20-v5.xls
File size is 24093 B
2017%20County%20Health%20Ranking%20Wyoming%20Data%20-v5.xls
File size is 24092 B
2017%20County%20Health%20Rankings%20Wyoming%20Data%20-

File size is 24098 B
2018%20County%20Health%20Ranking%20Wyoming%20Data%20-%20v4_0.xls
File size is 24097 B
2018%20County%20Health%20Rankings%20Wyoming%20Data%20-v4_0.xls
File size is 24095 B
2018%20County%20Health%20Ranking%20Wyoming%20Data%20-v4_0.xls
File size is 24094 B
2018%20County%20Health%20Rankings%20Wyoming%20Data%20-%20v4_1.xls
File size is 24098 B
2018%20County%20Health%20Ranking%20Wyoming%20Data%20-%20v4_1.xls
File size is 24097 B
2018%20County%20Health%20Rankings%20Wyoming%20Data%20-v4_1.xls
File size is 24095 B
2018%20County%20Health%20Ranking%20Wyoming%20Data%20-v4_1.xls
File size is 24094 B
2018%20County%20Health%20Rankings%20Wyoming%20Data%20-%20v3.xls
File size is 24096 B
2018%20County%20Health%20Ranking%20Wyoming%20Data%20-%20v3.xls
File size is 24095 B
2018%20County%20Health%20Rankings%20Wyoming%20Data%20-v3.xls
File size is 24093 B
2018%20County%20Health%20Ranking%20Wyoming%20Data%20-v3.xls
File size is 24092 B
2018%20County%20Health%20Rankings%20Wyoming%20Data%20-